In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations 

import matplotlib.pyplot as plt
import json
import networkx as nx 
import argparse
import config
import os 

import re



# 1, Evaluation

We usally run >3 tests to get avergaed peformance over random network initalization and random communication dropouts

In [3]:
#data_set = 'scannet'
#scene = 'scene0000'
#last_mesh_id = 773 # scene0106 
#last_mesh_id = 1858 # scene0000

data_set = 'Replica'
scene = 'room1'
last_mesh_id = 665 # room1

#exp_name = 'DiNNO_D50'
#exp_name = 'Ours_D50'
#exp_name = 'DSGD_D50'
#exp_name = 'DSGT_D50'
#exp_name = 'CADMM_D50'
exp_name = 'Ours_With_NARUTO'

num_of_agents = 3
number_of_trials = 3


For scannet, we cull centralized mesh, and use it as the ground truth mesh  
simply copy outputs to terminal to run them

In [ ]:
#last_centralized = 2019
last_centralized = 5577 # scene0000
#last_centralized = 2323 # scene0106
print(f'INPUT_MESH=output/scannet/{scene}_00/Centralized/agent_0/mesh_track{last_centralized}.ply')
print(f'python cull_mesh.py --config configs/scannet/{scene}.yaml --input_mesh $INPUT_MESH --remove_occlusion --gt_pose')

INPUT_MESH=output/scannet/office1_00/Centralized/agent_0/mesh_track5577.ply
python cull_mesh.py --config configs/scannet/office1.yaml --input_mesh $INPUT_MESH --remove_occlusion --gt_pose


Cull meshes and get evaluation scores  
Copy outputs into ```evaluation.sh```  
First remove previous log file  ```rm -rf screen.log```.  
Then run ```script screen.log``` to log terminal output. You need to reactivate conda env after running this.   
Then run evaluation by ```bash evaluation.sh```

In [4]:
if data_set == 'scannet':
    for agent_id in range(num_of_agents):
        for i in range(number_of_trials):
            print('\n')
            print(f'INPUT_MESH=output/scannet/{scene}_00/{exp_name}_{i+1}/agent_{agent_id}/mesh_track{last_mesh_id}.ply')
            print(f'python cull_mesh.py --config configs/scannet/{scene}.yaml --input_mesh $INPUT_MESH --remove_occlusion --gt_pose')
            print('\n')
            print(f'REC_MESH=output/scannet/{scene}_00/{exp_name}_{i+1}/agent_{agent_id}/mesh_track{last_mesh_id}_cull_occlusion.ply')
            print(f'GT_MESH=output/scannet/{scene}_00/Centralized/agent_0/mesh_track{last_centralized}_cull_occlusion.ply')
            print(f'python eval_recon.py --rec_mesh $REC_MESH --gt_mesh $GT_MESH -3d')
            print('\n')
elif data_set =='Replica':
    for agent_id in range(num_of_agents):
        for i in range(number_of_trials):
            print('\n')
            print(f'INPUT_MESH=output/Replica/{scene}/{exp_name}_{i}/agent_{agent_id}/mesh_track{last_mesh_id}.ply')
            print(f'VIRT_CAM_PATH=eval_data/Replica/{scene}/virtual_cameras')
            print(f'python cull_mesh.py --config configs/Replica/{scene}.yaml --input_mesh $INPUT_MESH --remove_occlusion --virtual_cameras --virt_cam_path $VIRT_CAM_PATH --gt_pose')
            print('\n')
            print(f'REC_MESH=output/Replica/{scene}/{exp_name}_{i}/agent_{agent_id}/mesh_track665_cull_virt_cams.ply')
            print(f'GT_MESH=eval_data/Replica/{scene}/gt_mesh_cull_virt_cams.ply')
            print(f'python eval_recon.py --rec_mesh $REC_MESH --gt_mesh $GT_MESH --dataset_type Replica -3d') # add '-2d' to get depth L1 loss
            print('\n')
else:
    print('WARNING: unsupported dataset')



INPUT_MESH=output/Replica/room1/Ours_With_NARUTO_0/agent_0/mesh_track665.ply
VIRT_CAM_PATH=eval_data/Replica/room1/virtual_cameras
python cull_mesh.py --config configs/Replica/room1.yaml --input_mesh $INPUT_MESH --remove_occlusion --virtual_cameras --virt_cam_path $VIRT_CAM_PATH --gt_pose


REC_MESH=output/Replica/room1/Ours_With_NARUTO_0/agent_0/mesh_track665_cull_virt_cams.ply
GT_MESH=eval_data/Replica/room1/gt_mesh_cull_virt_cams.ply
python eval_recon.py --rec_mesh $REC_MESH --gt_mesh $GT_MESH --dataset_type Replica -3d




INPUT_MESH=output/Replica/room1/Ours_With_NARUTO_1/agent_0/mesh_track665.ply
VIRT_CAM_PATH=eval_data/Replica/room1/virtual_cameras
python cull_mesh.py --config configs/Replica/room1.yaml --input_mesh $INPUT_MESH --remove_occlusion --virtual_cameras --virt_cam_path $VIRT_CAM_PATH --gt_pose


REC_MESH=output/Replica/room1/Ours_With_NARUTO_1/agent_0/mesh_track665_cull_virt_cams.ply
GT_MESH=eval_data/Replica/room1/gt_mesh_cull_virt_cams.ply
python eval_recon.py --r

First stop screeg.log by running ```exit```.  
The code below fetch evaluation results from ```screen.log```
I think a better way is using ```nohup bash evaluation.sh >screen.log 2>&1 &```

In [3]:
def extract_metrics(filename="screen.log"):
  """
  Extracts accuracy, completion, and completion ratio values from a log file.

  Args:
    filename: The path to the log file.

  Returns:
    A tuple containing three NumPy arrays: accuracy_values, completion_values, 
    and completion_ratio_values.
  """
  accuracy_values = []
  completion_values = []
  completion_ratio_values = []

  try:
    with open(filename, "r") as file:
      for line in file:
        # Use regular expressions to find the values
        accuracy_match = re.search(r"accuracy: (\d+\.?\d*)", line)
        completion_match = re.search(r"completion: (\d+\.?\d*)", line)
        completion_ratio_match = re.search(r"completion ratio: (\d+\.?\d*)", line)

        if accuracy_match:
          accuracy_values.append(float(accuracy_match.group(1)))
        if completion_match:
          completion_values.append(float(completion_match.group(1)))
        if completion_ratio_match:
          completion_ratio_values.append(float(completion_ratio_match.group(1)))

  except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
    return None, None, None

  return np.array(accuracy_values), np.array(completion_values), np.array(completion_ratio_values)

In [4]:
accuracy, completion, completion_ratio = extract_metrics()

print('Accuracy: {:.2f} ±{:.2f} '.format(np.mean(accuracy), np.std(accuracy)))
print('completion: {:.2f} ±{:.2f} '.format(np.mean(completion), np.std(completion)))
print('completion_ratio: {:.2f} ±{:.2f} '.format(np.mean(completion_ratio), np.std(completion_ratio)))

Accuracy: 40.59 ±0.92 
completion: 17.26 ±1.17 
completion_ratio: 26.59 ±1.24 
